In [1]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from pandas import Series, DataFrame
from sklearn.cross_validation import train_test_split
import matplotlib.pyplot as plt

In [2]:
train_fn = './Downloads/letter-recognition.data'
data_col = ['letter','x-box','y-box','width','high','onpix','x-bar','y-bar',
            'x2bar','y2bar','xybar','x2ybr','xy2br','x-ege','xegvy','y-ege','yegvx']
X = pd.read_csv(train_fn, sep=',', header=None, names=data_col,
                     skiprows=None, na_values='?', keep_default_na=False, engine='python')

In [3]:
y = X['letter']
y.head()

0    T
1    I
2    D
3    N
4    G
Name: letter, dtype: object

In [4]:
cols = list(X)
cols.pop(cols.index('letter'))
cols

['x-box',
 'y-box',
 'width',
 'high',
 'onpix',
 'x-bar',
 'y-bar',
 'x2bar',
 'y2bar',
 'xybar',
 'x2ybr',
 'xy2br',
 'x-ege',
 'xegvy',
 'y-ege',
 'yegvx']

In [5]:
X = X[cols]

In [6]:
X.head()

,x-box,y-box,width,high,onpix,x-bar,y-bar,x2bar,y2bar,xybar,x2ybr,xy2br,x-ege,xegvy,y-ege,yegvx
0,2,8,3,5,1,8,13,0,6,6,10,8,0,8,0,8
1,5,12,3,7,2,10,5,5,4,13,3,9,2,8,4,10
2,4,11,6,8,6,10,6,2,6,10,3,7,3,7,3,9
3,7,11,6,6,3,5,9,4,6,4,4,10,6,10,2,8
4,2,1,3,1,1,8,6,6,6,6,5,9,1,7,5,10


In [7]:
X_train = X[:16000]
X_test = X[16000:20000]
y_train = y[:16000]
y_test = y[16000:20000]

In [8]:
def models_N_weights(X, y, M, k):
    model = []
    model_weights = []
    training_errors = []
    
    N = X.shape[0]
    w = np.ones(N) / N

    for m in range(M):
        h = DecisionTreeClassifier(max_depth=1)
        h.fit(X, y, sample_weight=w)
        pred = h.predict(X)
        
        eps = w.dot(pred != y)
        alpha = (np.log((1 - eps)*(k - 1)) - np.log(eps)) / 2
        w_new = np.where(y == pred, w * np.exp(-alpha), w * np.exp(alpha))
        w = w_new / w_new.sum()
        
        model.append(h)
        model_weights.append(alpha)
    
    return [model, model_weights]


In [12]:
N = 10
frame = DataFrame(np.zeros([N,26]),columns=['A','B','C','D','E','F','G','H','I','J','k','L','M',
                                           'N','O','P','Q','R','S','T','U','V','W','X','Y','Z'])

In [11]:
frame.get_value(0, )

TypeError: get_value() takes at least 3 arguments (2 given)

In [25]:
def predict_joined_models(X, model, model_weights):
    N= X.shape[0]
    frame = DataFrame(np.zeros([N,26]),columns=['A','B','C','D','E','F','G','H','I','J','k','L','M',
                                           'N','O','P','Q','R','S','T','U','V','W','X','Y','Z'])
    #for j in range(len(model)):
    pred = model[-1].predict(X)
    for i in range(N):
        t = frame.get_value(i, pred[i])
        frame.set_value(i, pred[i], t + model_weights[-1])
            
    #获取每行最大数据的列名
    frame['max'] = frame.max(axis=1)
    frame['max_column'] = frame.T.idxmax()
    joined_model = frame.ix[:,'max_column']
 
    return joined_model

In [19]:
def error_func(y, y_hat):
    correct_pred = (np.array(y_hat) == np.array(y))
#     for i in range(len(y)):
#         if(y[i] == y_hat[i]):
#             correct_pred.append(1)
#         else:
#             correct_pred.append(0) 
    Err = 1 - float(sum(correct_pred))/len(correct_pred)
    return Err

In [18]:
len(np.unique(y))

26

In [15]:
M = 10
k = len(np.unique(y)) # 26
M_list = []
train_err_list = []
test_err_list = []

In [16]:
m=0

In [17]:
#for m in range(M):
model_fit = models_N_weights(X_train, y_train, m+1, k)

In [19]:
v = model_fit[0][-1].predict(X_test)

In [20]:
v

array(['T', 'T', 'T', ..., 'T', 'T', 'A'], dtype=object)

In [24]:
y_hat = predict_joined_models(X_train, model_fit[0], model_fit[1])

NameError: name 'predict_joined_models' is not defined

In [21]:
err = error_func(y_train, y_hat)
train_err_list.append(err)

y_hat = predict_joined_models(X_test, model_fit[0], model_fit[1])
err = error_func(y_test, y_hat)
test_err_list.append(err)
M_list.append(m)

KeyError: 0

In [22]:
plt.plot(M_list, train_err_list, c= 'red', linestyle='-')
plt.plot(M_list, test_err_list, c= 'green', linestyle='-')
plt.xlabel('number of weak learners')
plt.ylabel('Error')
plt.title('Error x Number of models')
plt.show()

ValueError: x and y must have same first dimension